In [1]:
from functools import partial
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="useer_agent", domain='nominatim.openstreetmap.org')
geocode = partial(geolocator.geocode, language="it", country_codes = "ita")

location = geolocator.geocode("Biblioteca comunale casalpusterlengo")
print(location.latitude, location.longitude)

45.1786532 9.64905444930788


In [2]:
import WazeRouteCalculator

from_address = '45.177015, 9.657982'
to_address = '45.159800, 9.554085'
region = 'EU'
route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
route_time, route_distance = route.calc_route_info()
print (f"Time {route_time} minutes, distance {route_distance} km.")

Time 14.466666666666667 minutes, distance 10.993 km.


In [3]:
from pandas_ods_reader import read_ods
import pandas as pd
path = "IndirizziBiblio_21-10-2019.ods"
df = read_ods(path, 'Questionari_normalizza')
df=df[df['Calendarizzata (sì/no)']=='sì']
df=df.astype({'Cap':'int32'})
df.head()

,Denominazione,Comune,Provincia,Indirizzo,Cap,Calendarizzata (sì/no)
0,Agnadello - Biblioteca Comunale,Agnadello,CR,"via Marconi, 6",26020,sì
1,Annicco - Biblioteca Comunale Remo Contardi,Annicco,CR,"piazza Giuseppe Garibaldi, 6",26021,sì
3,Bagnolo Cremasco - Biblioteca Comunale,Bagnolo Cremasco,CR,"piazza Roma, 3",26010,sì
4,Bonemerse - Biblioteca Comunale,Bonemerse,CR,"via Roma, 25",26040,sì
7,Camisano - Biblioteca Comunale,Camisano,CR,"via Trieste, 57",26010,sì


In [4]:
def getLocation(name):
    geolocator = Nominatim(user_agent="useer_agent")
    geocode = partial(geolocator.geocode, language="it")
    location =geolocator.geocode(name)
    return location.latitude, location.longitude
lat, lon = getLocation("Agnadello, marconi, 6")
#print(lat, lon)

In [5]:
def getName(row):
    name=row.Comune + ','
    try:
        name=name + row.Indirizzo[row.Indirizzo.lower().index('via')+3:len(row.Indirizzo)]
    except:
        try:
            name=name + row.Indirizzo[row.Indirizzo.lower().index('piazza')+6:len(row.Indirizzo)]
        except:
            try:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('piazzetta')+9:len(row.Indirizzo)]
            except:
                name=name + row.Indirizzo[row.Indirizzo.lower().index('vicolo')+6:len(row.Indirizzo)]
           
    return name

getLocation(getName(df.iloc[0]))

(45.4452793, 9.5563573)

In [30]:
lats = []
longs = []
not_found = []
for i in range(len(df)):
    try:
        name = getName(df.iloc[i])
        lat, lon = getLocation(name)
        lats.append(lat)
        longs.append(lon)
    except:
        not_found.append(name)
        continue

In [71]:
from tqdm.notebook import tqdm as tqdm

In [73]:
lat, lon=getLocation(getName(df.iloc[0]))
from_address=str(lat)+", "+str(lon)
double_error = False
times=[]
for i in tqdm(range(len(df))):
    region = 'EU'
    lat, lon=getLocation(getName(df.iloc[i]))
    to_address=str(lat)+", "+str(lon)
    try:
        route = WazeRouteCalculator.WazeRouteCalculator(from_address, to_address, region)
        route_time, route_distance = route.calc_route_info(real_time=False)
        double_error=False
    except:
        i=i-1
        if double_error==True:
            break
        double_error=True
    times.append((route_time, route_distance))
    #print (f"{i}-Time {route_time} minutes, distance {route_distance} km.")



In [85]:
t=[]
d=[]
for i in range(len(times)):
    t.append(times[i][0])
    d.append(times[i][1])

In [86]:
#NOT RUN
import csv
with open('times.csv', mode='w') as time_file:
    time_file = csv.writer(time_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    time_file.writerow(t)

with open('distances.csv', mode='w') as distance_file:
    distance_file = csv.writer(distance_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    distance_file.writerow(d)